# Run circuits in device emulators

# Implement Noise Model

In [1]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Kraus, SuperOp
from qiskit_aer import AerSimulator
from qiskit.tools.visualization import plot_histogram
from qiskit_aer.noise import (NoiseModel, QuantumError, ReadoutError,
    pauli_error, depolarizing_error, thermal_relaxation_error)
import numpy as np
from qiskit.tools.jupyter import *
import warnings
# warnings.filterwarnings('ignore')
from qiskit import *
import time
from qiskit.providers.aer.noise import NoiseModel
import qiskit.providers.aer.noise as noise
from qiskit.utils import QuantumInstance, algorithm_globals

In [2]:
from qiskit.providers.fake_provider import *

In [3]:
seeds = 170
algorithm_globals.random_seed = seeds
seed_transpiler = seeds
shot = int(np.floor(1800000 / 631))

In [4]:
shot

2852

In [42]:
shot * 631

1799612

## Hamiltonian conversion

By using SHVQE, we need to apply the inversal of Heisenberg part to the original Hamiltonian. First, load the original Hamiltonian from `Hamiltonian/OHhamiltonian.txt`.

In [5]:
with open('Hamiltonian/OHhamiltonian.txt', 'r') as file:
    pauli_text_lines = file.readlines()
paulis = []
weight = []
for line in pauli_text_lines:
    line = line.replace(' ', '')
    coeff, pauli_text_string = line.split("*")
    coeff = float(coeff)
    weight.append(coeff)
    # pauli_text_string = pauli_text_string[::-1] # TODO: this is a quick but wrong fix for qubits order
    pauli_text_string = pauli_text_string.replace('\n', '')
    paulis.append(pauli_text_string)

Then, we use `Stim` to apply the inversal of Heisenberg part to Hamiltonian. In our protocal, the Heisenberg part consists of `cz` gates, which commute with each other. The inversal of the Heisenberg part is itself.

In [6]:
import stim

Load Heisenberg circuit.

In [7]:
heisenberg_text_combined = ''

with open('saved_models/shvqe_clifford_ncz0_hei.qasm', 'r') as file:
    heisenberg_text_lines = file.readlines()
    heisenberg_text_combined = ''.join(heisenberg_text_lines[3:]) # omit the header

In [8]:
import re


# Define the regular expression pattern
pattern = r"q\[(\d+)\],q\[(\d+)\]"

# Define the replacement string
replacement = r"\1 \2"

# Use the sub() function to replace the matches
stim_text = re.sub(pattern, replacement, heisenberg_text_combined).replace(';', '')

Apply the Heisenberg circuit to each Pauli string.

In [9]:
%%time

heisenberg_circ = stim.Circuit(stim_text)
paulis_new = []

for p in paulis:
    ps  = stim.PauliString(p.replace('I', '_'))
    ps = ps.after(heisenberg_circ)
    paulis_new.append(ps.__str__().replace('_', 'I'))

CPU times: user 20.3 ms, sys: 735 µs, total: 21.1 ms
Wall time: 20.8 ms


Construct the new Hamiltonian as observable. The number of Pauli strings is not changed by Clifford.

In [10]:
from qiskit.quantum_info import SparsePauliOp

observable = SparsePauliOp(paulis_new, weight)
print(f">>> Observable size: {observable.size}")

>>> Observable size: 631


# Circuit Transpile and Execution

In [11]:
from qiskit import qasm2
from qiskit_aer.primitives import Estimator

circuit = qasm2.load('saved_models/shvqe_clifford_ncz0_sch.qasm')

Reverse qubits to respect qiskit convention.

In [12]:
circuit = circuit.reverse_bits()

In [13]:
# circuit.draw('mpl');

Transpile the circuit based on given system model from IBMQ_Montreal. **We use default transpiler from qiskit.**

In [14]:
system_model = FakeMontreal()

In [15]:
# transpiled_circuit = transpile(circuit, backend=system_model)

In [16]:
# transpiled_circuit.draw();

## Noiseless
Evaluate on the qiskit Estimator. Consider noiseless first.

In [17]:
estimator_noiseless = Estimator(
    backend_options = {
        # simulation options
        'method': 'statevector',
        'device': 'CPU',
        'max_parallel_threads' : 0,
        'max_parallel_shots' : 0,
        'statevector_parallel_threshold' : 5,
        'coupling_map' : system_model.configuration().coupling_map,
        'noise_model': None # noiseless
    },
    run_options = {
        'shots': shot,
        'seed': seeds,
    },
    transpile_options = {
        'seed_transpiler' : seed_transpiler,
    },
    skip_transpilation=False
)

In [18]:
%%time
job_noiseless = estimator_noiseless.run(circuit, observable)
result_noiseless = job_noiseless.result()
# print(f">>> {result}")
result_noiseless.values

CPU times: user 2h 4min 50s, sys: 20.5 s, total: 2h 5min 10s
Wall time: 1min 33s


array([-78.67793043])

In [19]:
def get_error_rate(val):
    ref_value = -74.38714627
    nuclear_repulsion_energy = 4.365374966545
    energy_with_nuclear_repulsion = val + nuclear_repulsion_energy
    error_rate = abs(abs(ref_value - energy_with_nuclear_repulsion) / ref_value * 100)
    return error_rate

In [20]:
get_error_rate(result_noiseless.values)[0]

0.10027378508814073

## Noise models

### Fake Kolkata

In [21]:
import pickle

In [22]:
with open('NoiseModel/fakekolkata.pkl', 'rb') as file:
    noise_model_fakekolkata = noise.NoiseModel.from_dict(pickle.load(file))

/tmp/ipykernel_24570/3864961774.py:2: DeprecationWarning: from_dict has been deprecated as of qiskit-aer 0.10.0 and will be removed no earlier than 3 months from that release date.
  noise_model_fakekolkata = noise.NoiseModel.from_dict(pickle.load(file))


In [23]:
noise_model_fakekolkata

<NoiseModel on ['', 'measure', 'sx', 'id', 'reset', 'x', 'cx']>

In [24]:
estimator_fakekolkata = Estimator(
    backend_options = {
        # simulation options
        'method': 'statevector',
        'device': 'CPU',
        'max_parallel_threads' : 0,
        'max_parallel_shots' : 0,
        'statevector_parallel_threshold' : 5,
        'coupling_map' : system_model.configuration().coupling_map,
        'noise_model': noise_model_fakekolkata
    },
    run_options = {
        'shots': shot,
        'seed': seeds,
    },
    transpile_options = {
        'seed_transpiler' : seed_transpiler,
    },
    skip_transpilation=False
)

In [25]:
%%time

job_fakekolkata = estimator_fakekolkata.run(circuit, observable)
result_fakekolkata = job_fakekolkata.result()
result_fakekolkata.values[0]

CPU times: user 1h 1min 4s, sys: 37.3 s, total: 1h 1min 42s
Wall time: 1min


-77.71795845630139

In [26]:
get_error_rate(result_fakekolkata.values)[0]

1.3907816499486456

### Fake Cairo

In [27]:
with open('NoiseModel/fakecairo.pkl', 'rb') as file:
    noise_model_fakecairo = noise.NoiseModel.from_dict(pickle.load(file))

/tmp/ipykernel_24570/1409486957.py:2: DeprecationWarning: from_dict has been deprecated as of qiskit-aer 0.10.0 and will be removed no earlier than 3 months from that release date.
  noise_model_fakecairo = noise.NoiseModel.from_dict(pickle.load(file))


In [31]:
estimator_fakecairo = Estimator(
    backend_options = {
        # simulation options
        'method': 'statevector',
        'device': 'CPU',
        'max_parallel_threads' : 0,
        'max_parallel_shots' : 0,
        'statevector_parallel_threshold' : 5,
        'coupling_map' : system_model.configuration().coupling_map,
        'noise_model': noise_model_fakecairo
    },
    run_options = {
        'shots': shot,
        'seed': seeds,
    },
    transpile_options = {
        'seed_transpiler' : seed_transpiler,
    },
    skip_transpilation=False
)

In [32]:
%%time

job_fakecairo = estimator_fakecairo.run(circuit, observable)
result_fakecairo = job_fakecairo.result()
result_fakecairo.values[0]

CPU times: user 1h 55s, sys: 36.1 s, total: 1h 1min 31s
Wall time: 1min 2s


-78.15958989384637

In [33]:
get_error_rate(result_fakecairo.values)[0]

0.7970884385677267

### Fake Montreal

In [34]:
with open('NoiseModel/fakemontreal.pkl', 'rb') as file:
    noise_model_fakemontreal = noise.NoiseModel.from_dict(pickle.load(file))

/tmp/ipykernel_24570/3624331980.py:2: DeprecationWarning: from_dict has been deprecated as of qiskit-aer 0.10.0 and will be removed no earlier than 3 months from that release date.
  noise_model_fakemontreal = noise.NoiseModel.from_dict(pickle.load(file))


In [35]:
noise_model_fakemontreal

<NoiseModel on ['', 'measure', 'sx', 'id', 'reset', 'x', 'cx']>

In [36]:
estimator_fakemontreal = Estimator(
    backend_options = {
        # simulation options
        'method': 'statevector',
        'device': 'CPU',
        'max_parallel_threads' : 0,
        'max_parallel_shots' : 0,
        'statevector_parallel_threshold' : 5,
        'coupling_map' : system_model.configuration().coupling_map,
        'noise_model': noise_model_fakemontreal
    },
    run_options = {
        'shots': shot,
        'seed': seeds,
    },
    transpile_options = {
        'seed_transpiler' : seed_transpiler,
    },
    skip_transpilation=False
)

In [37]:
%%time

job_fakemontreal = estimator_fakemontreal.run(circuit, observable)
result_fakemontreal = job_fakemontreal.result()
result_fakemontreal.values[0]

CPU times: user 1h 1min 14s, sys: 36.3 s, total: 1h 1min 51s
Wall time: 1min


-77.52843855466759

In [38]:
get_error_rate(result_fakemontreal.values)[0]

1.6455567167940766

## Duration

Obtain the Duration of Quantum Circuit

In [39]:
from qiskit import pulse

Do remember to set the optimization_level to 0 if your circuit is already transpiled!!!

In [40]:
transpiled_circuit = transpile(circuit, backend=system_model)

with pulse.build(system_model) as program:
  with pulse.transpiler_settings(optimization_level=0):
    pulse.call(transpiled_circuit)

In [41]:
program.duration

320